In [2]:
import pandas as pd
import json
import numpy as np
from bs4 import BeautifulSoup
import math


In [10]:
assist_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/full_dataset/skill_builder_data_corrected_collapsed.csv', encoding = "ISO-8859-1", low_memory=False)
pb_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/problem_bodies/ProblemBodies_23.csv', low_memory=False)

In [30]:
temp_df = pb_df[pb_df['curriculum']== 'Skill Builders']
temp_df[temp_df['grade_or_subject']== 'High School: Geometry']['problem_type'].unique()
temp1_df = temp_df[temp_df['grade_or_subject']== 'High School: Geometry']
temp1_df[temp1_df['problem_type']=='old_algebra']


temp2_df = temp_df[temp_df['grade_or_subject'].isin(['High School: Number and Quantity', 'High School: Algebra ', 
       'High School: Functions', 
       'High School: Statistics and Probability', 'High School: Geometry'])]


temp2_df ['unit'].unique()

x = temp2_df ['problem_set_name'].unique()

In [35]:
x
# x.shape

array(['(N.VM.C.8) Matrix Addition Skill Builder EX',
       '(N.VM.C.8) Matrix Rows and Columns Skill Builder',
       '(A.SSE.A.1a) Interpreting Parts of an Expression Skill Builder',
       '(N.VM.C.8) Matrix Multiplication Skill Builder',
       '(F.BF.A.1c) Composition of Functions Skill Builder',
       '(F.BF.A.2) Geometric and Arithmetic Sequences Skill Builder',
       '(F.BF.B.5) Logarithm Base Change Skill Builder',
       '(F.BF.B.4) Finding Inverse Functions Skill Builder',
       '(F.LE.A.4a) Solving ab^ct = d Skill Builder',
       '(F.BF.B.5) Basic Logarithm Manipulation Skill Builder',
       '(A.SSE.A.1a) Standard Form of a Polynomial Skill Builder',
       '(N.CN.A.2) Complex Numbers Skill Builder',
       '(F.TF.A.1) Converting Radians to Degrees Skill Builder',
       '(F.IF.B.4) Interpreting Graphs of Functions Skill Builder',
       '(F.BF.B.5) Intermediate Logarithm Manipulation Skill Builder',
       '(SS.MD.B.5) Finding Expected Value Skill Builder EX',
      

In [4]:
questions = assist_df['problem_id'].unique()
assist_df['skill_id'].unique()

skill_to_problems = {}
for skill_id in assist_df['skill_id'].unique():
  skill_to_problems[skill_id] = assist_df[assist_df['skill_id'] == skill_id]['problem_id'].unique()


In [5]:
with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/pb_subset/keyid2idx_assist2009_pb_subset.json', 'r') as f:
  keyid2idx_pb_subset = json.load(f)

In [6]:
problem_ids = list(keyid2idx_pb_subset['questions'].keys())
problem_ids_int = [int(x) for x in problem_ids]
subset_df_html_selected = pb_df[pb_df['problem_id'].isin(problem_ids_int)]
subset_df_html = subset_df_html_selected[['problem_id','problem_body']]

In [7]:
from bs4 import BeautifulSoup

def remove_tags(html):
    soup = BeautifulSoup(html, "html.parser")

    # Find and replace <img> tags with their 'alt' attribute
    for img in soup.find_all('img'):
        alt_text = img.get('alt', '')  # default to empty string if 'alt' is None
        if alt_text:
            # Create a new text node
            new_text = soup.new_string(" " + alt_text + " ")
            img.replace_with(new_text)
        else:
            # Remove the image if no alt text
            img.decompose()

    # Remove all script and style elements
    for tag in soup(['script', 'style']):
        tag.decompose()

    # Extract the text, cleaning up any excessive whitespace
    clean_text = ' '.join(soup.stripped_strings)
    return clean_text

# Example usage with a DataFrame column
# subset_df['problem_body'] = subset_df['problem_body'].apply(remove_tags)




#pass pb_df_subset['problem_body'] column into function remove_tags
subset_df_html['problem_body'] = subset_df_html['problem_body'].apply(remove_tags)

/Users/kiakarbasi/anaconda3/envs/kiapykt/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
subset_df_html.to_csv('//Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/problem_bodies/pb_assist2009_subset_preprocessed.csv', index=False)